# Transmisión de señales por un canal de comunicaciones
<p><code>Python en Jupyter Notebook</code></p>
<p>Creado por <code>Giancarlo Ortiz</code> para explicar los fundamentos de los <code>Sistemas de comunicaciones</code> en los cursos de telemática y redes de computadores.</p>

## Descripción
Este algoritmo, también llamado de caminos mínimos, es un método para resolver el problema del camino mas corto.

### Agenda
1. Grafos
1. Algoritmo
1. Código
1. Ejemplo

In [4]:
# Importar módulos al cuaderno
import math as m
import matplotlib.pyplot as plt

# Importar elementos de un módulo al cuaderno
from math import pi
from numpy import sin

## 1. Factores que afectan la transmisión  ✔

### 1.1. La atenuación 
la [Atenuación][alpha] es perdida de potencia de la señal.

$$ \alpha = 10 \cdot log_{10}\left(\frac{P_o}{P_i}\right) \\ $$

[alpha]:https://es.wikipedia.org/wiki/Atenuación



In [10]:
def attenuation(power_out:float, power_in:float):
    return round(20 * m.log10(power_out/power_in))

attenuation(70.7, 100)

-3

### 1.2. El ruido 
El [Ruido][noise] son las potencias parásitas que afectan la transmisión.

[noise]:https://es.wikipedia.org/wiki/Ruido_(comunicación)#

### 1.3. El eco
El [Eco][21_3] es reflexión de una parte de la potencia original.

[21_3]:https://es.wikipedia.org/wiki/P%C3%A9rdida_de_retorno